# Test advection-diffusion model


In [ ]:
# imports
import numpy as np
import cuqi
import sys
import matplotlib.pyplot as plt
from cuqi.distribution import Gaussian, JointDistribution, GMRF
from cuqi.geometry import Continuous2D
from cuqi.pde import TimeDependentLinearPDE
from cuqi.model import PDEModel
from custom_distribution import MyDistribution
from advection_diffusion_inference_utils import parse_commandline_args,\
    read_data_files,\
    create_domain_geometry,\
    create_PDE_form,\
    create_prior_distribution,\
    create_exact_solution_and_data,\
    set_the_noise_std,\
    sample_the_posterior,\
    create_experiment_tag,\
    plot_experiment,\
    save_experiment_data,\
    Args,\
    build_grids,\
    create_time_steps,\
    plot_time_series

#### Set up run arguments

In [ ]:
args = Args()
noise_level_list= ["fromDataVar" , "fromDataAvg", "avgOverTime", 0.1, 0.2]
args.noise_level = noise_level_list[3]
args.animal = 'm1'
args.ear = 'l'
args.num_ST = 4
args.inference_type = 'heterogeneous'
args.unknown_par_type = 'sampleMean'
args.unknown_par_value = ['m1:l:NUTS:constant:100.0:real:heterogeneous:1000:0.1:v:April22:2024:a::4:5@../../../Collab-BrainEfflux-Data/April_2x_2024_b']

tag = create_experiment_tag(args)
print(tag)



### Read the data

In [ ]:
times, locations, real_data, real_std_data = read_data_files(args)
# The left boundary condition is given by the data  
real_bc = abs(real_data.reshape([len(locations), len(times)])[0,:])
#real_bc_r = abs(real_data.reshape([len(locations), len(times)])[-1,:])

### Create the forward model

In [ ]:
#%% STEP 4: Create the PDE grid and coefficients grid
#----------------------------------------------------
# PDE and coefficients grids
L = locations[-1]*1.3
coarsening_factor = 5
n_grid_c = 20
grid, grid_c, grid_c_fine, h, n_grid = build_grids(L, coarsening_factor, n_grid_c)

#%% STEP 5: Create the PDE time steps array
#------------------------------------------
tau_max = 30*60 # Final time in sec
cfl = 4 # The cfl condition to have a stable solution
         # the method is implicit, we can choose relatively large time steps 
tau = create_time_steps(h, cfl, tau_max)

#%% STEP 6: Create the domain geometry
#-------------------------------------
G_c = create_domain_geometry(grid_c, args.inference_type)

# STEP 7: Create the PDE form
#----------------------------
PDE_form = create_PDE_form(real_bc, grid, grid_c, grid_c_fine, n_grid, h, times,
                           args.inference_type)

# STEP 8: Create the CUQIpy PDE object
#-------------------------------------
PDE = TimeDependentLinearPDE(PDE_form,
                             tau,
                             grid_sol=grid,
                             method='backward_euler', 
                             grid_obs=locations,
                             time_obs=times) 

# STEP 9: Create the range geometry
#----------------------------------
G_cont2D = Continuous2D((locations, times))

# STEP 10: Create the CUQIpy PDE model
#-------------------------------------
A = PDEModel(PDE, range_geometry=G_cont2D, domain_geometry=G_c)

#### Create the prior distribution

In [ ]:
# STEP 11: Create the prior distribution
#---------------------------------------
#x = create_prior_distribution(G_c, args.inference_type)
x1 = GMRF(np.ones(G_c.par_dim-1)*np.sqrt(300),
            0.2,
            bc_type='neumann')
x2 = Gaussian(0.5, 0.3**2)
x = MyDistribution([x1, x2], geometry=G_c )

#from cuqi.distribution import GMRF
#x = GMRF(np.ones(G_c.par_dim)*np.sqrt(300), 0.2, geometry=G_c, bc_type='neumann')

Ns = 10
samples_x = x.sample(Ns)
samples_x.plot(range(Ns), plot_par=True)
plt.figure()
samples_x.funvals.plot(range(Ns))

In [ ]:
# Plot prior CI
x.sample(1000).plot_ci(95)

In [ ]:
# Test different prior
x_alternative = cuqi.distribution.GMRF(np.ones(G_c.par_dim)*np.sqrt(100), 2, geometry=G_c, bc_type='neumann')
x_alternative.sample(1000).plot_ci(95)

In [ ]:
### test
A

#### Create and plot exact data

In [ ]:
a = np.sqrt(0.1)
x_true, exact_data = create_exact_solution_and_data(A, args.unknown_par_type, args.unknown_par_value, a=a)
plot_time_series(times, locations, exact_data.reshape([len(locations), len(times)]))


In [ ]:
PDE.assemble(x_true)
sol, _ = PDE.solve()
print(sol.shape)

plt.plot(grid, sol)

len(tau)
plt.figure()
x_true.funvals.plot()

#### Create the data distribution

In [ ]:
#%% STEP 13: Create the data distribution
#----------------------------------------
# First, illustrate how different noise levels setups affect the std
print("Standard deviation values")
plt.figure()
for item in noise_level_list:
    s_noise_temp = set_the_noise_std(
        args.data_type, item, exact_data,
        None, real_std_data, G_cont2D)
    print('\n**noise_level option**:', item)
    print('**STD of the noise**:')
    print(s_noise_temp)
    # plot noise level
    if isinstance(s_noise_temp, np.ndarray):
        plt.plot(s_noise_temp, label=str(item))
    else:
        plt.plot(s_noise_temp*np.ones(G_cont2D.par_dim), label=str(item))
plt.legend()
plt.title('Noise level')
plt.xlabel('data point index i\n data point for the same location are grouped together')
plt.ylabel('std of the noise')

# Second, set the noise level
s_noise = set_the_noise_std(args.data_type, args.noise_level, exact_data,
                                None, real_std_data, G_cont2D)
y = Gaussian(A(x), s_noise**2, geometry=G_cont2D)

#### Samples of noisy data and plot the noisy data and the noise

In [ ]:
noisy_data = y(x=x_true).sample()

fig, ax = plt.subplots(1, 3, figsize=(14, 3))
plt.sca(ax[0])
plot_time_series(times, locations, exact_data.reshape([len(locations), len(times)]), plot_legend=False)
plt.title('Exact data')

plt.sca(ax[1])
plot_time_series(times, locations, noisy_data.reshape([len(locations), len(times)]), plot_legend=False)
plt.title('Noisy data')

diff = noisy_data - exact_data
plt.sca(ax[2])
lines, labels = plot_time_series(times, locations, diff.reshape([len(locations), len(times)]), plot_legend=False)
plt.title('Difference')
# Noise to signal ratio
print('Noise to signal ratio (synthitic data): ', np.linalg.norm(diff)/np.linalg.norm(noisy_data))
print('STD to signal ratio (real data): ', np.linalg.norm(real_std_data)/np.linalg.norm(real_data))

fig2 = plt.figure()
# turn off axis
plt.axis('off')
plt.legend(lines, labels, loc='center',ncol=3)
#figure size
fig2.set_size_inches(14, 1)



In [ ]:
# Create the Bayesian model
#--------------------------
BP = cuqi.problem.BayesianProblem(x, y)
BP.set_data(y=noisy_data)
BP.posterior.enable_FD()
x0 = np.ones(G_c.par_dim)*10
x0[-1] = 0
map = BP.MAP(x0=x0)

Demo forward with a = 1 , a = 2, a = 3 (instability in the last case)

In [ ]:
r=2
BP_test.posterior.gradient(np.zeros(BP_test.posterior.dim))

In [ ]:
# Create the Bayesian model
#--------------------------
import cuqi
import numpy as np
gaussian = cuqi.distribution.Gaussian(np.zeros(2)+0.5, 0.3**2)
x_test = cuqi.distribution.UserDefinedDistribution(logpdf_func=gaussian.logpdf, sample_func=gaussian.sample, dim=2)
A_test = cuqi.model.Model(forward=lambda x_test: x_test**2, range_geometry=2, domain_geometry=2)
y_test = cuqi.distribution.Gaussian(A_test(x_test), 0.1**2)
x_test_true = x_test.sample(1)
y_test_data = y_test(x_test=x_test_true).sample()
BP_test = cuqi.problem.BayesianProblem(x_test, y_test)
BP_test.set_data(y_test=y_test_data)
BP_test.posterior.enable_FD()
BP_test.UQ()


In [ ]:
print(x_test_true)
print(map)

In [ ]:
map.plot(label='MAP')
x_true.plot(label='True solution', marker='x')
plt.plot(x0, 'o', label='Initial guess')
mean = np.concatenate((x1.mean.flatten(), x2.mean))
plt.plot(mean, '.', label='Prior mean')
plt.legend(loc='lower center')

In [ ]:
min(x_true[:-1])**2
max(x_true[:-1])**2

a = Pec * min_diffusion / L
Pec_min = a * L / min_diffusion
Pec_max = a * L / max_diffusion
Pec_min = 0.9**2 * 1900 / 21 = 72.9?
Pec_max = 0.9**2 * 1900 / 200 = 8.19?


a = 0.9 micro m/s

L = ~1900 micro m

min_diffusion = 21 micro m^2/s

http://calliope.dem.uniud.it/CLASS/ING-AMB/ade.pdf

0.01 to get pec of 0.1

0.01 * 2000 / (15)**2 = 20 /225 = 0.0889
a = sqrt(0.01) = 0.1

CA only inference 


In [ ]:
2/(0.3*0.01)

In [ ]:
np.sqrt(3000)